<a href="https://colab.research.google.com/github/Ro-netizen004/PALM_Lab/blob/main/Copy_of_Zero_shot_benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Assignment details:
Rodela: Qwen 2.5-1.5B Datasets to run eval on: MMLU - GSM8k - commonsenseQA

#Install the required libraries

In [ ]:
!pip install torch transformers datasets evaluate --quiet
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00


##Import the required libraries

We import PyTorch, the deep learning framework. PyTorch provides the tensor operations and GPU acceleration required to run large language models like Qwen efficiently. It’s the computation engine behind the model.

AutoTokenizer converts text into token IDs the model can understand, and can decode tokens back into text. AutoModelForCausalLM loads a causal language model for generating text (predicting the next token in a sequence). These classes allow us to easily load and use pretrained models from Hugging Face.

The datasets library provides an easy way to load benchmark datasets from Hugging Face. It handles downloading, formatting, and splitting datasets automatically. We use it to load the questions or inputs for zero-shot evaluation.

The evaluate library provides ready-to-use metrics for evaluating model predictions.
Examples: accuracy, F1 score, BLEU, ROUGE.
We use it to measure model performance in a standardized way.

Pandas is a library for manipulating tabular data. We use it to organize model inputs, outputs, and evaluation scores into dataframes. This makes it easy to analyze, visualize, and save results.



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import evaluate
import pandas as pd
!pip install tqdm
from tqdm import tqdm



##Load the Model

We store the Hugging Face model ID in a variable. This specifies which large language model we want to use for zero-shot evaluation. Using a variable makes it easy to switch to other models later without changing multiple lines of code.

The tokenizer converts text into tokens (numbers) that the model can process, and vice versa. Each model has its own tokenizer because vocabulary and tokenization rules differ between models. This step ensures the input text is in the correct format for Qwen.

AutoModelForCausalLM loads a causal language model (like GPT or Qwen) suitable for text generation.

device_map="auto" automatically places the model on the best available hardware (GPU or CPU), which is convenient for large models.

torch_dtype=torch.bfloat16 loads the model in bfloat16 precision, reducing memory usage and speeding up inference on GPUs like the H100 or A100, while maintaining accuracy.



In [ ]:
model_name = "Qwen/Qwen2.5-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",            # automatically place model on GPU
    torch_dtype=torch.bfloat16    # H100-optimized precision
)

model.eval()  # set model to inference mode


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

##Load the dataset, and test with 50 samples first to ensure pipeline works.

This line loads a subset of the GSM8K benchmark dataset for evaluation:

"gsm8k" – the dataset name. GSM8K (Grade School Math 8K) is a set of math word problems used for testing reasoning in language models.

"main" – the dataset configuration; it selects the standard version of GSM8K (some datasets have multiple configurations or styles).

split="test[:50]" – specifies the portion of the dataset to load:

test → the evaluation split of the dataset

[:50] → only the first 50 examples, useful for quick testing or debugging before running the full test set.

In [ ]:
dataset = load_dataset("gsm8k", "main", split="test")

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

##Define zero-shot generation function
This function generates zero-shot answers from a language model for any dataset.

It formats the input text using a prompt template (prompt_template) so the model knows what to generate.

The prompt is tokenized and moved to the same device as the model (CPU/GPU).

torch.no_grad() ensures efficient inference without gradients, and model.generate() predicts tokens deterministically up to max_tokens.

The generated tokens are decoded into text, the prompt is removed, and an optional clean_func can further process the answer.

Returns the final answer as a string, ready for evaluation.

In [ ]:
def generate_answer_generic(
    input_text,
    prompt_template="{text}\nAnswer:",
    max_tokens=128,
    clean_func=None
):
    """
    Generate a zero-shot answer from a language model for any dataset.

    Parameters:
    - input_text: str, the input question/text from the dataset
    - prompt_template: str, a template to format the input
        e.g., "{text}\nAnswer:" or "Question: {text} Options: {options}\nAnswer:"
    - max_tokens: int, maximum tokens to generate
    - clean_func: callable, optional function to clean/process model output

    Returns:
    - answer: str, model-generated answer
    """

    # Format prompt using template
    """
    prompt_template.format(text=input_text) constructs the final prompt by inserting
    the provided input text into a predefined template string. The template contains a {text} placeholder which
    is replaced with the actual input at runtime, allowing the same prompt structure to be reused with different inputs.
    """

    prompt = prompt_template.format(text=input_text)

    # Tokenize and move to model device
    #Converts the prompt text into token IDs that the model can read.
    #return_tensors="pt" → outputs PyTorch tensors.
    #.to(model.device) → ensures the tensors are on the same device as the model (CPU/GPU).

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output (inference mode, deterministic)
    with torch.no_grad(): # torch.no_grad() → disables gradient computation because we are not training, which saves memory and speeds up inference.
        outputs = model.generate( #model.generate() → the model predicts tokens continuing from the prompt.
            **inputs,
            max_new_tokens=max_tokens, # max_new_tokens=max_tokens → stop after generating the specified number of tokens.
            do_sample=False # do_sample=False → deterministic output (same input → same output).
        )

    # Decode tokens to text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove prompt part
    answer = decoded.replace(prompt, "").strip()

    # Optional custom cleaning
    if clean_func:
        answer = clean_func(answer)

    return answer


##Run Zero Shot Inference
This loop goes through each example in the dataset, extracts the question and ground-truth answer, and generates a zero-shot prediction using the model. The question, model prediction, and reference answer are stored in separate lists (questions, predictions, references) for later analysis, and each example is printed with a separator so you can quickly inspect how the model is performing.

In [ ]:
questions = dataset["question"]
references = dataset["answer"]

batch_size = 50  # adjust based on your GPU/memory
all_predictions = []

for i in tqdm(range(0, len(questions), batch_size)):
    batch_questions = questions[i:i+batch_size]

    # Generate predictions for this batch
    batch_predictions = [generate_answer_generic(
            input_text=q,
            prompt_template="Q: {text}\nA:",  # GSM8K format
            max_tokens=128,
            clean_func=lambda x: x.split("\n")[0]  # optional
        ) for q in batch_questions]  # replace with your model call

    all_predictions.extend(batch_predictions)


  0%|          | 0/27 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_

In [ ]:
def extract_numeric_answer(text):
    # Remove everything except the last number in the string
    import re
    numbers = re.findall(r"\d+", text)
    if numbers:
        return int(numbers[-1])  # take the last number as the answer
    else:
        return None


##Simple accuracy evaluation
How the Zero-Shot Accuracy is Computed

Predictions and References:
predictions are the model’s answers for each question.
references are the correct answers from the GSM8k dataset.

Extracting Numeric Answers:

Since the model output may contain extra text (like explanations), we first extract the numeric answer from both the prediction and the reference using the extract_numeric_answer() function.

Only pairs where both the prediction and reference have a valid numeric answer are kept for evaluation.

Rounding for Exact Match:

All extracted numbers are rounded to the nearest integer.

This ensures small decimal differences do not affect the exact-match accuracy.

Computing Accuracy:

The cleaned and rounded predictions are compared to the rounded references.

Accuracy is calculated as the fraction of predictions that exactly match the reference:

In [ ]:
clean_predictions = []
clean_references = []

for p, r in zip(all_predictions, references):
    pred_num = extract_numeric_answer(p)
    ref_num = extract_numeric_answer(r)
    if pred_num is not None and ref_num is not None:
        clean_predictions.append(round(pred_num))
        clean_references.append(round(ref_num))

# -------------------------------
# Compute zero-shot accuracy manually
# -------------------------------
print(clean_predictions)
print(clean_references)
correct = sum(p == r for p, r in zip(clean_predictions, clean_references))
total = len(clean_predictions)
accuracy = correct / total

print(f"Zero-shot accuracy: {accuracy:.4f}")

# -------------------------------
# Save results to CSV
# -------------------------------
df = pd.DataFrame({
    "question": questions,
    "prediction": all_predictions,
    "reference": references
})

df.to_csv("gsm8k_qwen_zero_shot.csv", index=False)
print("Saved: gsm8k_qwen_zero_shot.csv")


[18, 3, 0, 540, 6, 83, 260, 160, 1, 460, 366, 694, 12, 100, 221, 300, 57500, 7, 1, 15, 17, 7, 10, 375, 2, 50, 28, 25, 104, 110, 280, 35, 40, 23, 9, 15, 1, 86, 4, 8, 1200, 32, 1350, 0, 44, 243, 800, 80, 30, 140, 2, 15, 120, 33, 14, 3, 83, 57, 187, 15, 1430, 105000, 147, 300, 36, 96, 595, 36, 60, 975, 60, 221, 76, 1, 5, 2727, 100, 9, 70, 12, 17, 623, 600, 15, 4, 22, 660, 8000, 24, 225, 28, 4, 545455, 228, 80, 3, 16, 25, 40, 175, 75, 12, 140, 500, 20, 354, 7, 1, 28, 45, 2, 24, 25, 6, 2, 142857, 360, 4, 56000, 240, 27, 47, 50, 1, 4, 82, 120, 900, 0, 30, 940, 60, 13, 720, 55, 6, 29, 140, 70, 20, 466, 140, 5, 20, 333333, 2375, 45, 14, 16, 1, 50, 4, 208, 170, 14, 1220, 34, 15, 16, 13, 92, 50, 15, 77, 5, 2, 18, 120, 150, 1190, 5, 6000, 5, 5, 100, 200, 122, 130, 20, 160, 21, 151, 6, 30, 10, 106, 400, 25, 120, 5, 4, 6, 10, 12, 120, 6277, 4, 15000, 55, 367200, 100, 8, 62, 130, 860, 4200, 76, 993151, 60, 42, 5, 500, 2, 44, 220, 15, 48, 251, 70, 90, 140, 20000, 180, 9, 32, 0, 60, 21, 2, 50, 75, 12,

##MMLU Dataset

In [ ]:
# Load the MMLU test split
dataset = load_dataset("mmlu", "all", split="test")  # "all" loads all subjects

questions = dataset["question"]  # the question text
references = dataset["answer"]   # correct option (like "A", "B", ...)
options_list = dataset["choices"]  # multiple choice options, e.g., ["A", "B", "C", "D"]
